# raC_pso_gene.ipynb

Hyperparameter search on the reach_analysis_C model using either Particle Swarm Optimization, a genetic algorithm, or a combinatin of both using two machines.

When running both algorithms, one machine maintains a population evolved through the PSO algorithm, whereas the second machine uses a genetic algorithm. They share their configurations through a shared file system.

-Sergio Verduzco-Flores  
May 2022

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from datetime import datetime
from multiprocessing import Pool
from pathos.multiprocessing import ProcessingPool

/home/z/projects/draculab


In [2]:
%load_ext Cython

In [3]:
%%cython
from draculab import *

In [4]:
%cd /home/z/projects/draculab/notebook/spinal/
from net_builders import *
%cd ../..

/home/z/projects/draculab/notebook/spinal
/home/z/projects/draculab


In [ ]:
# Utility methods, ranges, search configuration parameters

focus_params = True # whether to focus mutations on a specific set of parameters
max_evals = 8 # maximum number of times to evaluate a configuration
target_fitness = 0.02 # stop if fitness reaches this value

# we have 3 separate network configurations to explore
class sim_config(Enum):
    STATIC_CONN = 1
    C_LEARNS = 2
    M_LEARNS = 3
    
test = sim_config['M_LEARNS'] # select the configuration

# Specify paramters and ranges
ranges = {"A__C_lrate" : {"low": 0.1, "high": 30., "default":15.},
          "A__C_w_max_frac" : {"low": .05, "high": 1., "default":.3},
          "A__C_w_sum" : {"low": .5, "high": 10., "default":.8},
          "A__M_lrate" : {"low": 0.1, "high": 30., "default":20.},
          "A__M_w_max_frac" : {"low": .05, "high": 1., "default":.4},
          "A__M_w_sum" : {"low": .5, "high": 10., "default":2.},
          "AL_thresh" : {"low": -.1, "high": 1.5, "default":.7},
          "b_e": {"low": .5, "high": 5., "default":1.},
          "C__C_antag": {"low": 0.1, "high": 3., "default":1.5},
          "C__C_p_antag": {"low": 0., "high": 1.5, "default":.25},
          "C__C_p_syne": {"low": 0., "high": 1., "default":.3},
          "C__C_syne": {"low": 0., "high": 2.5, "default":1.},
          "C_adapt_amp": {"low": 0., "high": 5., "default":.0},
          "C_cid" : {"low": 0.1, "high": 2., "default":.15},
          "C_sigma" : {"low": 0., "high": 1., "default":.4},
          "C_slope" : {"low": 0.5, "high": 4., "default":2.3},
          "C_tau" : {"low": 0.01, "high": .4, "default":.2},
          "C_tau_slow" : {"low": 2., "high": 20., "default":5.},
          "CE_thresh" : {"low": 0., "high": 4., "default":1.},
          "CE__CI_w": {"low": 0., "high": 2.5, "default":.5},
          "CI__CE_w": {"low": -2.5, "high": 0, "default":-1.8},
          "CI_slope" : {"low": 1., "high": 5., "default":3.8},
          "CI_tau" : {"low": 0.01, "high": .2, "default":.025},
          "CI_thresh" : {"low": 0., "high": 4., "default":1.5},  
          "g_e_03": {"low": 15., "high": 25., "default":20.},  
          "g_e_factor": {"low": 0.5, "high": 4., "default":3.},
          "II_g_03": {"low": 2., "high": 9., "default":3.},
          "k_pe_e": {"low": 16., "high": 25., "default":20.},
          "k_se_e": {"low": 16., "high": 25., "default":20.},
          "M__AL_lrate" : {"low": .1, "high": 600., "default":500.},
          "M__AL_w_sum": {"low": 0.5, "high": 8., "default": 1.5},
          "M_adapt_amp": {"low": 0., "high": 5., "default":.0},
          "M__C_lrate" : {"low": .1, "high": 600., "default":500.},
          "M__C_w_sum": {"low": 0.5, "high": 8., "default": 3.},
          "M__M_w": {"low": 0., "high": -3., "default":-.5},   # must set to zero when permutting M
          "M_cid": {"low": 0.05, "high": 2., "default": 1.1},
          "M_des_out_w_abs_sum": {"low": 0.5, "high": 4., "default": 2.},
          "M_tau": {"low": .005, "high":.2, "default":.03},
          "M_slope": {"low": .5, "high":4., "default":2.7},
          "M_thresh" : {"low": 0., "high": 3.5, "default":.5},
          "M_sigma" : {"low": 0., "high": 1., "default":.4},
          "SF_slope": {"low": .5, "high":4.5, "default":4.},
          "SF_thresh_03": {"low": .2, "high": 1.1, "default":.55},
          "SPF__M_lrate": {"low": .1, "high": 600., "default":500.},
          "SPF__M_w_sum": {"low": .5, "high": 5., "default":2.5},
          "SPF_des_out_w_abs_sum": {"low": .5, "high": 5., "default":1.5},
          "SPF__SPF_w": {"low": -3., "high": 0., "default":-1.77}
         }

def set_value(name, value):
    """ Set a static value for a parameter in the 'ranges' dictionary. """
    ranges[name]['low'] = value
    ranges[name]['high'] = value
    ranges[name]['default'] = value

if test.value == sim_config.STATIC_CONN.value:
    rga_on_M = False # whether to use rga_21 connections on SPF__M
    M_noise = False # whether M units are noisy (use euler_maru integrator)
    C_noise = False # whether C units are noisy (use euler_maru integrator)
    M__C_rand_w = False #True # whether to randomly intialize weights for the M__C connections
    old_M__C = False # when using hand-set M__C connections  (e.g. M__C_rand_w=False)
                 # old_M__C=False uses permutted M with no negative connections
    M__M_conns = True # antagonist inhibition in M
    rand_targets = False # Random or radial targets
    set_value('M__C_lrate', 0.1) # remove M__C plasticity
    set_value('M__AL_lrate', 0.1)
    set_value('M__M_w', 0.)
    set_value('C_adapt_amp', 0.)
    set_value('M_adapt_amp', 0.)
#     main_pars = {'A__C_w_sum', 'A__M_w_sum', 'AL_thresh', 'CI_slope', 'CI_thresh', 'CE_thresh', 
#                  'M_slope', 'M_thresh', 'M__C_w_sum', 'M__AL_w_sum', 'C_slope', 'C_tau', 'M_tau',
#                  'SF_slope', 'g_e_03', 'II_g_03', 'SPF__M_w_sum'}
    main_pars = {'A__C_w_max_frac', 'A__M_w_max_frac', 'A__C_w_sum', 'A__M_w_sum',
                'CE_thresh', 'CI_thresh', 'C_tau', 'M_thresh'}
elif test.value == sim_config.C_LEARNS.value:
    rga_on_M = False # whether to use rga_21 connections on SPF__M
    M_noise = False # whether M units are noisy (use euler_maru integrator)
    C_noise = True # whether C units are noisy (use euler_maru integrator)
    M__C_rand_w = True # whether to randomly intialize weights for the M__C connections
    old_M__C = False
    M__M_conns = True # antagonist inhibition in M
    rand_targets = True # Random or radial targets
    set_value('M_adapt_amp', 0.)
    set_value('SPF__M_lrate', 0.)
    main_pars = {'M__M_w', 'CE_thresh', 'CI_thresh', 'C_sigma', 'C_cid', 'M_slope',
                 'C_slope', 'CI_slope', 'M_thresh', 'M__C_w_sum', 'C_adapt_amp',
                 'C_tau'}
elif test.value == sim_config.M_LEARNS.value:
    rga_on_M = True # whether to use rga_21 connections on SPF__M
    M_noise = True # whether M units are noisy (use euler_maru integrator)
    C_noise = False # whether C units are noisy (use euler_maru integrator)
    M__C_rand_w = False # whether to randomly intialize the M__C connections
    old_M__C = False # when using hand-set M__C connections  (e.g. M__C_rand_w=False)
                 # old_M__C=False uses permutted M with no negative connections
    M__M_conns = False # antagonist inhibition in M
    rand_targets = True # Random or radial targets
    set_value('M__C_lrate', 0.)
    set_value('M__AL_lrate', 0.)
    set_value('C_adapt_amp', 0.)

    main_pars = {'CE_thresh', 'CI_thresh', 'M_sigma', 'SPF__M_lrate', 'M_cid', 
                 'C_slope', 'CI_slope', 'M_thresh', 'M_slope', 'M__C_w_sum', 
                 'SPF_des_out_w_abs_sum', 'SPF__M_w_sum', 'SPF__M_lrate', 'C_adapt_amp'}
    ranges['M_thresh']['default'] = (ranges['SPF_des_out_w_abs_sum']['default'] + 
                                     ranges['SPF__M_w_sum']['default']) / 4.
else:
    raise ValueError('Invalid simulation configuration')


#par_list = [name for name in ranges] # ordered list with names of the parameters
par_list = list(ranges.keys())
# parameters to focus on
# main_pars.update({'A__C_lrate', 'A__C_w_max_frac', 'A__C_w_sum', 'AL_thresh', "A__M_w_max_frac",
#                   "A__M_w_sum", "A__M_lrate", "g_e_03", "SF_thresh_03", "M_tau", 
#                   "M_des_out_w_abs_sum", "C_slope", "C_tau", "CI_slope", "CI_tau", "C_tau_slow"})
main_pars = list(main_pars)
if focus_params:
    par_list = main_pars

/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for th

In [6]:
# Functions for the genetic algorithm and for creating initial parameters

def mutate(cfg, name_list=par_list):
    """ Mutate a single parameter of the given configuration. 
    
        Args:
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] =  l + (h-l)*np.random.random()
    
def soft_mutate(cfg, ma, name_list=par_list):
    """ Soft-mutate a single parameter of the given configuration.
    
        A 'soft mutation' keeps the mutated value close to the original value.
        The maximum amplitude of the mutation is given by the 'ma' argument.
        
        Args:
            ma : float in (0,1]. Max. amplitude as fraction of the parameter's range 
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] = max(l, min(h, cfg[par_name] + ma * (h-l) * (np.random.random()-0.5)))

In [ ]:
# Create the initial population: method 1

# create variations of the parameters we want to investigtate
pop_size = 90 # number of configurations in the population

default_dic = {}
for name in list(ranges.keys()):
    default_dic[name] = ranges[name]['default']

pop = [default_dic.copy() for _ in range(pop_size)]

n_def = 3 # how many will have the default values
        
# fill variations in both directions
chg_name = ["low", "default", "high"] # auxiliary list
for ind in range(n_def, pop_size):
    for name in par_list:
        chg_dir = chg_name[np.random.randint(3)] 
        pop[ind][name] = 0.5 * (ranges[name][chg_dir] + ranges[name]["default"])
            
# Throw n_muts mutations, and n_soft_muts soft mutations
n_muts = 20
n_soft_muts = 10 # must have n_muts + n_soft_muts < pop_size-1
perm = np.random.permutation(range(1,pop_size)) # don't mutate the first element
for i in range(n_muts):
    mutate(pop[perm[i]])
for i in range(n_muts,n_muts+n_soft_muts):
    soft_mutate(pop[perm[i]], 0.2)
    
pop[10] = {'A__C_lrate': 15.0,
         'A__C_w_max_frac': 0.3,
         'A__C_w_sum': 0.8,
         'A__M_lrate': 20.0,
         'A__M_w_max_frac': 0.7,
         'A__M_w_sum': 1.5,
         'AL_thresh': 0.7,
         'b_e': 1.0,
         'C__C_antag': 1.5,
         'C__C_p_antag': 0.25,
         'C__C_p_syne': 0.3,
         'C__C_syne': 1.0,
         'C_adapt_amp': 0.0,
         'C_cid': 0.15,
         'C_sigma': 0.2,
         'C_slope': 1.4,
         'C_tau': 0.30000000000000004,
         'C_tau_slow': 12.5,
         'CE_thresh': 1.0,
         'CE__CI_w': 0.5,
         'CI__CE_w': -1.8,
         'CI_slope': 3.8,
         'CI_tau': 0.0175,
         'CI_thresh': 1.5,
         'g_e_03': 17.5,
         'g_e_factor': 3.0,
         'II_g_03': 6.0,
         'k_pe_e': 20.0,
         'k_se_e': 20.0,
         'M__AL_lrate': 0.1,
         'M__AL_w_sum': 1.5,
         'M__C_lrate': 0.1,
         'M__C_w_sum': 3.0,
         'M__M_w': -0.5,
         'M_cid': 1.55,
         'M_des_out_w_abs_sum': 2.0,
         'M_tau': 0.115,
         'M_slope': 2.7,
         'M_thresh': 0.5,
         'SF_thresh_03': 0.375,
         'SPF__M_lrate': 500.0,
         'SPF__M_w_sum': 2.5,
         'SPF_des_out_w_abs_sum': 1.5,
         'SPF__SPF_w': -1.77,
         'fitness': 0.02124628616637105,
         'n_evals': 1,
         't_pres': 40,
         'par_heter': 0.01}

In [7]:
# Create the initial population using an MN star

# 1) Obtain star
def MNstar(N, M, mu, theta, Dt, maxiters=1e4, miniters=100, V=None):
    """Returns M N-dimensional maximally-separated vectors.
    
        Args:
            N: dimension of the vectors
            M: number of vectors
            mu: viscous friction coefficient
            theta: simulation will continue until max. change < theta
            Dt: simulation time step
        Optional Args:
            maxiters: max number of steps
            V: initial vectors
            miniters: minimum number of sim steps (to get things moving)
        Returns:
            V: an NxM numpy array with the separated vectors
            c_max: a measure of how much the vectors were changing
            step: simulation step where the function terminated
    """
    c_max = theta + 1
    if V is None:
        V = 1. - 2.*np.random.random((N,M))
    V = V / np.linalg.norm(V, axis=0) # normalizing
    init_V = V.copy()
    dV = np.zeros_like(V)
    P = np.zeros((M,M))
    step = 0
    while (c_max > theta and step < maxiters) or step < miniters:
        ddV = np.zeros_like(V)
        for j in range(M):
            for k in range(j+1,M):
                P[j,k] = max(min(np.dot(V[:,j],V[:,k]), 1.-1e-10), -1.+1e-10) 
                q = min(1. / max((np.arccos(P[j,k])**2) * (1. - P[j,k]*P[j,k]), 1e-10), 1.e2)
                ddV[:,j] = ddV[:,j] + q * (P[j,k] * V[:,j] - V[:,k])
                ddV[:,k] = ddV[:,k] + q * (P[j,k] * V[:,k] - V[:,j])
                
        c_max = 0.
        for j in range(M):
            ddV[:,j] = ddV[:,j] - mu * dV[:,j]
            dV[:,j] = dV[:,j] + Dt * ddV[:,j]
            v_old = V[:,j].copy()
            V[:,j] = V[:,j] + Dt * dV[:,j]
            V[:,j] = V[:,j] / np.linalg.norm(V[:,j])
            c = max(min(np.dot(V[:,j], v_old), 1.-1e-10), -1.+1e-10)
            dV[:,j] = (c*V[:,j] - v_old) / Dt
            if 1.-c > c_max: c_max = 1.-c
        step += 1
        
    return V, c_max, step

pop_size = 90 # number of configurations in the population

# A star is born
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
star, _, _  = MNstar(len(par_list), pop_size-1, .2, 1e-6, .1)   #!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# 2) Obtain initial population and center
default_dic = {}

# Using the default values in 'ranges' as the center
#-----------------------------------------------------------
# for name in list(ranges.keys()):
#     default_dic[name] = ranges[name]['default']

# center = default_dic
#-----------------------------------------------------------
# If you want to use a center different from the default
# load a parameter configuration
# fname = '/home/z/Dropbox (OIST)/saves/gene_2022-05-17'
# with (open(fname, "rb")) as f:
#     pop_load = pickle.load(f)
#     f.close()
# center = pop_load[0]
#-----------------------------------------------------------

center = {'A__C_lrate': 22.5,
         'A__C_w_max_frac': 0.3154013926076185,
         'A__C_w_sum': 2.0121150990439123,
         'A__M_lrate': 26.168940205264438,
         'A__M_w_max_frac': 0.06860586458007485,
         'A__M_w_sum': 0.8899137448068103,
         'AL_thresh': 1.1026808777262314,
         'b_e': 1.0,
         'C__C_antag': 1.8349307795916774,
         'C__C_p_antag': 0.15778174934184203,
         'C__C_p_syne': 0.17882009260444504,
         'C__C_syne': 0.5036133455082225,
         'C_adapt_amp': 3.3,
         'C_cid': 0.3734541257204699,
         'C_sigma': 0.6325506984405911,
         'C_slope': 1.6392718094309338,
         'C_tau': 0.07514574987464283,
         'C_tau_slow': 11.0,
         'CE_thresh': 1.9558164246281846,
         'CE__CI_w': 0.5,
         'CI__CE_w': -1.8,
         'CI_slope': 3.8905582976717508,
         'CI_tau': 0.017882988750483223,
         'CI_thresh': 2.5,
         'g_e_03': 22.37496854909262,
         'g_e_factor': 3.0,
         'II_g_03': 7.460435647390968,
         'k_pe_e': 20.0,
         'k_se_e': 20.0,
         'M__AL_lrate': 0.0,
         'M__AL_w_sum': 2.8637790596264376,
         'M__C_lrate': 0.0,
         'M__C_w_sum': 2.729567159544233,
         'M__M_w': -0.7425261391037636,
         'M_cid': 0.9438697747358124,
         'M_des_out_w_abs_sum': 2.52147222263484,
         'M_tau': 0.046822234340659984,
         'M_slope': 3.0140943843745664,
         'M_thresh': 2.5,
         'M_sigma': 0.45378209920376733,
         'SF_thresh_03': 0.7510206904604764,
         'SPF__M_lrate': 503.2750199447509,
         'SPF__M_w_sum': 3.0,
         'SPF_des_out_w_abs_sum': 3.0,
         'SPF__SPF_w': -1.77,
         'fitness': 0.23272883455182444,
         'n_evals': 2,
         't_pres': 40.0,
         'par_heter': 0.01,
         'SF_slope': 3.052353930622451,
         'SPF_M_lrate': 500.0}

#-----------------------------------------------------------
pop = [center.copy() for _ in range(pop_size)]

# 3) Expand configuration around the center
for conf_idx, conf in enumerate(pop[1:]):
    for par_idx, par_name in enumerate(par_list):
        move = 0.5 * star[par_idx, conf_idx]
        dir_tag = 'low' if move < 0. else 'high'
        conf[par_name] = center[par_name] + abs(move) * (ranges[par_name][dir_tag] - center[par_name])


In [ ]:
# Create the initial population: method 2

# Load the results from a previous run
#fname = '/home/z/projects/draculab/saves/v3ft3p2ph2_pop_2021-05-28__10_53'
fname = '/home/z/Dropbox (OIST)/saves/gene_2021-06-17'
with (open(fname, "rb")) as f:
    prev_pop1 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/gene_2021-06-24'
with (open(fname, "rb")) as f:
    prev_pop2 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/pso_2021-06-28'
with (open(fname, "rb")) as f:
    prev_pop3 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/pso_2021-06-24'
with (open(fname, "rb")) as f:
    prev_pop4 = pickle.load(f)
    f.close()
    
fname = '/home/z/Dropbox (OIST)/saves/gene_2021-07-19'
with (open(fname, "rb")) as f:
    prev_pop5 = pickle.load(f)
    f.close()
    
fname = '/home/z/Dropbox (OIST)/saves/pso_2021-07-19'
with (open(fname, "rb")) as f:
    prev_pop6 = pickle.load(f)
    f.close()
# If the results are from a run with fewer parameters
# fill it with the default values.
for cfg in prev_pop1 + prev_pop2 + prev_pop3 + prev_pop4:
    for name in ranges:
        if not name in cfg:
            cfg[name] = ranges[name]['default']


In [ ]:
# Mix configurations from the two methods
pop[0:15] = prev_pop1[:20]
pop[15:30] = prev_pop2[:20]
pop[30:45] = prev_pop3[:20]
pop[45:60] = prev_pop4[:20]
pop[60:75] = prev_pop5[:20]
pop[75:90] = prev_pop6[:20]

# Best configuration from June 11th
cfg11 = {'A__M_w_max_frac': 0.9,
         'A__M_w_sum': 2.0,
         'C_adapt_amp': 4.4,
         'C_cid': 0.12,
         'C_sigma': 0.67,
         'M_cid': 1.1,
         'M_des_out_w_abs_sum': 3.0,
         'g_e_factor': 3.0,
         'C_slope': 2.,
         'C_thresh': 1.1,
         'C_tau': 0.26,
         'C_tau_slow': 10.0,
         'A__M_lrate': 8.0,
         'AL_thresh': 0.55,
         'b_e': 1.0,
         'C__C_antag': 1.5,
         'C__C_p_antag': 0.25,
         'C__C_p_syne': 0.3,
         'C__C_syne': 1.0,
         'CE__CI_w': 0.5,
         'CI__CE_w': -1.3,
         'M__C_lrate': 300.0,
         'M__C_w_sum': 2.3,
         'M__M_w': -0.5,
         'SPF__SPF_w': -1.5,
         'fitness': None,
         'n_evals': 0,
         't_pres': 40,
         'par_heter': 0.01,
         'CI_slope': 2.5,
         'CI_thresh': 1.4,
         'CI_tau': 0.15,
         'g_e_03': 20.0,
         'II_g_03': 3.0,
         'M_tau': 0.01,
         'SF_thresh_03': 0.4}
cfg_syne={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.3,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.55,
     'b_e': 1.5,
     'C__C_antag': 1.5,
     'C__C_p_antag': 0.25,
     'C__C_p_syne': 0.28,
     'C__C_syne': 1.,
     'C_adapt_amp': 0.4,
     'C_cid': 0.15,
     'C_sigma': 0.5,
     'C_slope': 2.1,
     'C_tau': 0.23,
     'C_tau_slow': 40.0,
     'C_thresh': 0.93,
     'CE__CI_w': 0.5,
     'CI__CE_w': -1.3,
     'g_e_03': 25.,
     'CI_slope': 3.6,
     'CI_tau': 0.017,
     'CI_thresh': 1.4,
     'g_e_factor': 3.2,
     'II_g_03': 3.16,
     'M__C_lrate': 200.,
     'M__C_w_sum': 2.5,
     'M__M_w': 0.0,
     'M_cid': 1.1,
     'M_des_out_w_abs_sum': 3.,
     'M_tau': 0.024,
     'SF_thresh_03': 0.63,
     'SPF__SPF_w': -1.5,
     'fitness': None,
     'n_evals': 0,
     't_pres': 40.,
     'par_heter': 0.01}

cfg_std={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.34,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.56,
     'b_e': 1.,
     'C__C_antag': 1.6,
     'C__C_p_antag': 0.15,
     'C__C_p_syne': 0.26,
     'C__C_syne': 1.1,
     'C_adapt_amp': 0.0,
     'C_cid': 0.17,
     'C_sigma': 0.5,
     'C_slope': 2.25,
     'C_tau': 0.24,
     'C_tau_slow': 2.0,
     'C_thresh': 1.14,
     'CE__CI_w': 0.39,
     'CI__CE_w': -1.8,
     'g_e_03': 20.,
     'CI_slope': 3.9,
     'CI_tau': 0.06,
     'CI_thresh': 1.37,
     'g_e_factor': 3.,
     'II_g_03': 2.73,
     'M__C_lrate': 500.0,
     'M__C_w_sum': 3.28,
     'M__M_w': 0.0,
     'M_cid': 1.,
     'M_des_out_w_abs_sum': 1.87,
     'M_tau': 0.012,
     'SF_thresh_03': 0.59,
     'SPF__SPF_w': -1.6,
     'fitness': None,
     'n_evals': 0,
     't_pres': 40.,
     'par_heter': 0.01}

pop[68] = cfg_std
pop[78] = cfg_syne
pop[88] = cfg11

# If configurations have fewer parameters, fill them with the default values.
# Also, homogeneize parameters not in par_list
for cfg in pop:
    for name in ranges:
        if (not name in cfg or 
            (focus_params and not name in par_list)):
            cfg[name] = cfg11[name]

In [8]:
# reset fitness and number of evaluations
for dic in pop:
    if not 'fitness' in dic or not 'nevals' in dic:
        dic['fitness'] = None # average fitness value
        dic['n_evals'] = 0  # number of times fitness has been evaluated

In [9]:
# Set search parameters present in the configurations
for cfg in pop:
    cfg['t_pres'] = 40.
    cfg['par_heter'] = 0.01

In [10]:
# print used configuration
for dic in pop[0:4]:
    print('{',end='')
    for name in dic.keys():
        print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name in net_conf:
#             print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name != 'fitness' or dic['fitness'] != None:
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__C_lrate':22.5, 'A__C_w_max_frac':0.3154013926076185, 'A__C_w_sum':2.0121150990439123, 'A__M_lrate':26.168940205264438, 'A__M_w_max_frac':0.06860586458007485, 'A__M_w_sum':0.8899137448068103, 'AL_thresh':1.1026808777262314, 'b_e':1.0, 'C__C_antag':1.8349307795916774, 'C__C_p_antag':0.15778174934184203, 'C__C_p_syne':0.17882009260444504, 'C__C_syne':0.5036133455082225, 'C_adapt_amp':3.3, 'C_cid':0.3734541257204699, 'C_sigma':0.6325506984405911, 'C_slope':1.6392718094309338, 'C_tau':0.07514574987464283, 'C_tau_slow':11.0, 'CE_thresh':1.9558164246281846, 'CE__CI_w':0.5, 'CI__CE_w':-1.8, 'CI_slope':3.8905582976717508, 'CI_tau':0.017882988750483223, 'CI_thresh':2.5, 'g_e_03':22.37496854909262, 'g_e_factor':3.0, 'II_g_03':7.460435647390968, 'k_pe_e':20.0, 'k_se_e':20.0, 'M__AL_lrate':0.0, 'M__AL_w_sum':2.8637790596264376, 'M__C_lrate':0.0, 'M__C_w_sum':2.729567159544233, 'M__M_w':-0.7425261391037636, 'M_cid':0.9438697747358124, 'M_des_out_w_abs_sum':2.52147222263484, 'M_tau':0.0468222343

{'A__C_lrate': 22.5,
 'A__C_w_max_frac': 0.3154013926076185,
 'A__C_w_sum': 2.0121150990439123,
 'A__M_lrate': 26.168940205264438,
 'A__M_w_max_frac': 0.06860586458007485,
 'A__M_w_sum': 0.8899137448068103,
 'AL_thresh': 1.1026808777262314,
 'b_e': 1.0,
 'C__C_antag': 1.8349307795916774,
 'C__C_p_antag': 0.15778174934184203,
 'C__C_p_syne': 0.17882009260444504,
 'C__C_syne': 0.5036133455082225,
 'C_adapt_amp': 3.3,
 'C_cid': 0.3734541257204699,
 'C_sigma': 0.6325506984405911,
 'C_slope': 1.6392718094309338,
 'C_tau': 0.07514574987464283,
 'C_tau_slow': 11.0,
 'CE_thresh': 1.9558164246281846,
 'CE__CI_w': 0.5,
 'CI__CE_w': -1.8,
 'CI_slope': 3.8905582976717508,
 'CI_tau': 0.017882988750483223,
 'CI_thresh': 2.5,
 'g_e_03': 22.37496854909262,
 'g_e_factor': 3.0,
 'II_g_03': 7.460435647390968,
 'k_pe_e': 20.0,
 'k_se_e': 20.0,
 'M__AL_lrate': 0.0,
 'M__AL_w_sum': 2.8637790596264376,
 'M__C_lrate': 0.0,
 'M__C_w_sum': 2.729567159544233,
 'M__M_w': -0.7425261391037636,
 'M_cid': 0.943869774

In [12]:
# A function that evaluates the fitness of a given configuration
def eval_config(cfg):
    """ Returns the error for a network with a given configuration.

        Args:
            cfg : a configuration dictionary.
        Returns:
            error : A float calculated from the sum of activities in the SPF layer.
    """
    np.random.seed() # will try to get a seed from /dev/urandom
    if cfg['n_evals'] > max_evals: # if the fitness has been evaluated "enough" times. See cell below...
        return cfg['fitness']
    t_pres = cfg['t_pres']
    
    # obtain a network with the given configuration
    net, pops_dict, hand_coords, m_idxs, pds = net_from_cfg(cfg,
                                                       t_pres = t_pres,
                                                       par_heter = cfg['par_heter'],
                                                       set_C_delay = False,
                                                       rand_targets = rand_targets,
                                                       track_weights = False,
                                                       track_ips = False,
                                                       C_noise = C_noise,
                                                       M__C_rand_w = M__C_rand_w,
                                                       M_noise = M_noise,
                                                       rga_on_M = rga_on_M,
                                                       rdc_on_M = False,
                                                       rot_SPF = False,
                                                       M__M_conns = M__M_conns,
                                                       old_M__C = old_M__C)
    # run the network
    run_time = 500 #2. * 5. * 12 * t_pres #400 # 1000.
    #start_time = time.time()
    times, data, plant_data  = net.flat_run(run_time)
    #print('Execution time is %s seconds' % (time.time() - start_time))

    # calculate average error in last half of reaching
    P = pops_dict['P']
    arm_activs = plant_data[P]
    plant = net.plants[P]
    # modified copy-paste of plt.upd_ip_impl
    q1 = arm_activs[:,0]
    q2 = arm_activs[:,2]
    q12 = q1+q2
    c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
    c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                    c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
    coord_idxs = np.floor(times/t_pres).astype(int)
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time

    error_time = run_time - round(run_time/2.)
    error_idx = int(round(error_time/net.min_delay))
    hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
    hand_error_integ = hand_error[error_idx:].sum()
    avg_hand_error = hand_error_integ / (hand_error.size - error_idx)

    #return hand_error_integ
    return avg_hand_error

---
# Genetic algorithm  

---

In [13]:
# A function to produce offspring by crossing individuals
par_names = list(ranges.keys()) #list(pop[0].keys()) # list with all parameter names

def create_offspring(cfg1, cfg2, par_list=par_names):
    """ Given 2 configurations, return 2 offspring from random swapping.
    
        To produce offspring, first we choose one split point in the
        dictionary. The first offspring has the values of cfg1 up to that
        point, and cfg2 afterwards. The second offspring has the cfg2 values
        up to the split point, and cfg1 afterwards. Since the dictionaries are
        not ordered, we use a parameter list to set the split point.
    
        Args:
            cfg1, cfg2: parameter dictionaries
            par_list: list with the keys in cfg1, cfg2 (unless focus_params)
        Returns:
            cfg3, cfg4: dictionaries from swapping values in cfg1, cfg2
    """
    if focus_params:
        par_list = main_pars
    sp = np.random.randint(len(par_list))# split point as an index to par_list
    cfg3 = cfg1.copy()
    cfg4 = cfg2.copy()
    for i in range(sp, len(par_list)):
        cfg3[par_list[i]] = cfg2[par_list[i]]
        cfg4[par_list[i]] = cfg1[par_list[i]]
    return cfg3, cfg4

In [14]:
####################################
###### The genetic algorithm ######
####################################

#pop = pop[0:15] # limit pop size for debugging
n_mates = 30 # number of individuals to mate at each generation (even number)
max_gens = 30 # maximum number of generations
n_soft_mut = 10 # number of individuals to soft-mutate per generation
r_soft_mut = 0.2 # relative amplitude of soft mutations
n_mut = 8 #number of individuals to mutate per generation
n_procs = 30 # number of processes to use for fitness evaluation
n_save = 3 #number of individuals to protect from replacement and mutation
use_pso = True # whether to insert configurations from the PSO algorithm
repl_num = 8 #how many individuals to replace with PSO configurations

# setting name for file where parameters will be stored
path = "/home/z/Dropbox (OIST)/saves/"
#path = "/home/z/projects/draculab/saves/"
fname1 = "gene"
fname2 = datetime.now().strftime('%Y-%m-%d')
fname = path + fname1 + "_" + fname2

pop_len = len(pop)

for gen in range(max_gens):
    start_time = time.time()
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
    ######## parallel version, python multiprocessing module
#     with Pool(n_procs) as p:
#         fits = list(p.map(eval_config, pop))
#         p.close()
#         p.join()
    ######## parallel version, pathos
    with ProcessingPool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
    #print(fits)
    # 1.2) update the average fitness values
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        #print(nr)
        #print(fits[idx])
        #print(cfg['fitness'])
        if nr > 0:
            if nr <= max_evals:
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
    #---------------------------------------------------------------------------------
    # 1.3) Inserting configurations from the PSO algorithm
    if use_pso:
        pso_name = path + "pso_" + fname2
        try:
            with open(pso_name, 'rb') as f:
                pso_pop = pickle.load(f)
                f.close()
            pop[-repl_num:] = pso_pop[:repl_num]
            ext_pop = pop + pso_pop[repl_num:]
            print("Mixed pso and gene pops!!!")
        except IOError:
            if gen > 2: # if the pso algorithm should likely be done
                from warnings import warn
                warn('population ' + pso_name + 'could not be imported',
                     UserWarning)
            ext_pop = pop
    #---------------------------------------------------------------------------------
    # 2) Sort according to fitness. Lowest error first.
    if use_pso:
        pop = sorted(ext_pop, key=lambda d: d['fitness'])[:pop_len]
    else:
        pop = sorted(pop, key=lambda d: d['fitness'])
    # 2.1) Save current generation
    with open(fname, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
    # 2.2) A quick message
    print("Generation %d evaluated. Best fitness: %.3f"%(gen,pop[0]['fitness']))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if pop[0]['fitness'] < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    # 3) mate and replace
    # 3.1) Select individuals to be replaced with probability proportional to error
    fits.sort() # sort the fitnesses (now in the same order as pop)
    fits = np.array(fits)
    fits = fits/fits.sum() # normalize fitnesses so they add to 1
    cumsum_fits = fits[:] # cumsum_fits[i] = sum(fits[:i])
    for i in range(1,len(cumsum_fits)):
        cumsum_fits[i] = cumsum_fits[i-1] + cumsum_fits[i]
    repl_list = [] # list with indexes of individuals to be replaced
    while len(repl_list) < n_mates:
        min_r = cumsum_fits[n_save] # don't replace the first n_save individuals
        r = min_r + (1.-min_r) * np.random.random()
        candidate = n_save
        for i in range(n_save, len(fits)):
            if cumsum_fits[i] > r:
                break
            candidate += 1
        if candidate in repl_list:
            continue
        else:
            repl_list.append(candidate)
    print("to replace: ", end='')
    print(repl_list)
    # 3.2) Arrange individuals in random pairs
    perm = np.random.permutation(n_mates) # this will do 
    # 3.3) mate
    new_pops = []
    for i in range(int(np.floor(n_mates/2))):
        off1, off2 = create_offspring(pop[perm[2*i]], pop[perm[2*i+1]])
        new_pops.append(off1)
        new_pops.append(off2)
    # 3.4) replace
    for i, cfg in enumerate(new_pops):
        pop[repl_list[i]] = cfg
    # 4) mutate
    # 4.1) soft mutations
    for _ in range(n_soft_mut):
        idx = np.random.randint(len(pop))
        if idx < n_save:
            copy = pop[idx].copy()
            soft_mutate(copy, r_soft_mut)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            soft_mutate(pop[idx], r_soft_mut)
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
    # 4.2) mutations
    # 4.2.1) select individuals to mutate
    # sq_fits = fits*fits
    #cumsum_sq_fits = fits * fits # cumsum_sq_fits[i] = sum(sq_fits[:i])
    cumsum_sq_fits = fits # proportional to fits, rather than its square
    cumsum_sq_fits = cumsum_sq_fits / cumsum_sq_fits.sum()
    for i in range(1,len(cumsum_sq_fits)):
        cumsum_sq_fits[i] = cumsum_sq_fits[i-1] + cumsum_sq_fits[i]
    mut_list = [] # list with indexes of individuals to be mutate
    while len(mut_list) < n_mut:
        r = np.random.random()
        candidate = 0
        for i in range(len(fits)):
            if cumsum_sq_fits[i] > r:
                break
            candidate += 1
        if candidate in mut_list:
            continue
        else:
            mut_list.append(candidate)
    print("to mutate: ", end='')
    print(mut_list)
    for idx in mut_list:
        if idx < n_save:
            copy = pop[idx].copy()
            mutate(copy)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            mutate(pop[idx])
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
            
    print('generation %d finished in %s seconds' % (gen, time.time() - start_time))

    

/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for th

Generation 0 evaluated. Best fitness: 0.067
Mean fitness = 0.337
to replace: [26, 78, 36, 83, 82, 81, 48, 39, 84, 72, 73, 68, 87, 41, 23, 59, 29, 18, 66, 74, 77, 14, 85, 60, 44, 40, 5, 89, 54, 65]
to mutate: [52, 62, 86, 66, 50, 68, 59, 61]
generation 0 finished in 8061.2244782447815 seconds
Generation 1 evaluated. Best fitness: 0.089
Mean fitness = 0.330
to replace: [27, 40, 77, 50, 46, 47, 78, 69, 22, 61, 75, 65, 68, 86, 43, 42, 81, 66, 72, 83, 10, 80, 58, 82, 88, 7, 89, 54, 62, 48]
to mutate: [80, 81, 53, 63, 66, 27, 67, 15]
generation 1 finished in 8895.017029047012 seconds
Generation 2 evaluated. Best fitness: 0.077
Mean fitness = 0.347
to replace: [75, 59, 88, 83, 31, 16, 62, 72, 44, 70, 56, 25, 77, 79, 27, 89, 22, 40, 78, 73, 12, 6, 38, 50, 35, 15, 58, 21, 39, 87]
to mutate: [89, 66, 68, 64, 75, 73, 82, 85]
generation 2 finished in 8010.893112421036 seconds


<ipython-input-14-27e942c14c48>:66: UserWarning: population /home/z/Dropbox (OIST)/saves/pso_2022-05-26could not be imported
  warn('population ' + pso_name + 'could not be imported',


Generation 3 evaluated. Best fitness: 0.106
Mean fitness = 0.337
to replace: [85, 6, 72, 48, 77, 15, 86, 18, 12, 51, 78, 87, 4, 83, 81, 82, 30, 80, 76, 66, 42, 55, 34, 57, 22, 23, 61, 89, 84, 28]
to mutate: [70, 73, 76, 79, 75, 65, 41, 37]
generation 3 finished in 8104.395681142807 seconds
Generation 4 evaluated. Best fitness: 0.049
Mean fitness = 0.320
to replace: [88, 33, 79, 89, 46, 23, 63, 12, 26, 27, 82, 58, 22, 43, 54, 14, 11, 64, 77, 35, 10, 9, 38, 39, 30, 69, 52, 50, 13, 67]
to mutate: [55, 26, 88, 87, 45, 81, 86, 60]
generation 4 finished in 8121.166483879089 seconds
Generation 5 evaluated. Best fitness: 0.040
Mean fitness = 0.302
to replace: [67, 88, 40, 80, 65, 18, 79, 41, 86, 60, 24, 82, 36, 68, 7, 75, 69, 66, 89, 45, 17, 83, 77, 49, 11, 70, 51, 72, 85, 34]
to mutate: [21, 68, 88, 73, 81, 80, 76, 35]
generation 5 finished in 8159.40554857254 seconds
Mixed pso and gene pops!!!
Generation 6 evaluated. Best fitness: 0.044
Mean fitness = 0.315
to replace: [55, 64, 72, 63, 80, 8

Process ForkPoolWorker-22:
Process ForkPoolWorker-17:
Process ForkPoolWorker-21:
Process ForkPoolWorker-11:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-30:
Process ForkPoolWorker-28:
Process ForkPoolWorker-4:
Process ForkPoolWorker-25:
Process ForkPoolWorker-5:
Process ForkPoolWorker-20:
Process ForkPoolWorker-8:
Process ForkPoolWorker-19:
Process ForkPoolWorker-9:
Process ForkPoolWorker-27:
Process ForkPoolWorker-14:
Process ForkPoolWorker-23:
Process ForkPoolWorker-2:
Process ForkPoolWorker-16:
Process ForkPoolWorker-10:
Process ForkPoolWorker-3:
Process ForkPoolWorker-15:
Process ForkPoolWorker-26:
Process ForkPoolWorker-13:
Process ForkPoolWorker-24:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-12:
Traceback (most recent call last):
Process ForkPool

KeyboardInterrupt: 

  File "/home/z/projects/draculab/plants/plants.py", line 2704, in tension_diff
    A = np.array([self.get_input_sum(time, i) for i in range(18)])
KeyboardInterrupt
KeyboardInterrupt
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for the acc_slow reset port
  warn('setting the 0 value for the acc_slow reset port')
/home/z/projects/draculab/requirements/requirements.py:1456: UserWarning: setting the 0 value for

In [ ]:
pop[0]

---
# Particle Swarm Optimization
---

In [ ]:
# Utility functions for the PSO algorithm
# These methods assume that 'par_list' and 'ranges' have been defined.
ref_cfg = pop[0] # a configuration with all values, including those that
                 # are not in par_list
    
def add_to_cfg(cfg, vel):
    """ Add a vector to the values of a configuration.
    
        Args:
            cfg : a configuration dictionary
            vel : Numpy array of values to be added
            
        The values in 'vel' are in the order of 'par_list'.
        Each entry in vel is a value in the [-mv, mv] interval, where mv
        is a parameter indicating the maximum velocity.
        
        For each parameter 'par_name' in 'par_list', the corresponding
        value in vel will be multiplied times
        (ranges[par_name]['high'] - ranges[par_name]['low'])
        before being added.
        
        If a value exceeds the limits set in 'ranges' it will be clipped.
        After updating cfg, its 'fitness' and 'n_evals' entries will be
        reset to their initial values.
    """
    for idx, name in enumerate(par_list):
        cfg[name] += vel[idx] * (
                       ranges[name]['high'] - ranges[name]['low'])
        cfg[name] = max( min(ranges[name]['high'], cfg[name]),
                           ranges[name]['low'])
    cfg['fitness'] = None
    cfg['n_evals'] = 0
        
def add_vel(vel, acc, mv=0.5):
    """ Add an acceleration to a velocity vector.
    
        Args:
            vel : velocity vector (numpy array).
            acc : acceleration vector (numpy array).
            mv : maximum velocity.
        Returns:
            vel + acc
            
        The entries of 'vel' and 'acc' are in the order of 'par_list'.
        If vel[i] + acc[i] > mv, or vel[i] + acc[i] < -mv, values will be
        clipped.
    """
    return np.maximum(np.minimum(vel + acc, mv), -mv)
    
def cfg_to_vec(cfg):
    """ Convert a configuration dictionary to a vector.
    
        Args:
            cfg: configuration dictionary.
        Returns:
            vec: a numpy array with the values of the configuration.
            
        The order of the values in 'vec' is that of 'par_list'.
        The 'cfg' entries for 'fitness', 'n_evals', 't_pres', and
        'par_heter' will be omitted in 'vec'.
    """
    vec = np.zeros(len(par_list))
    for idx, name in enumerate(par_list):
        vec[idx] = cfg[name]
    return vec

def vec_to_cfg(vec, fitness=None, n_evals=0, t_pres=None, par_heter=0.01):
    """ Convert a vector to a configuration dictionary.
    
        Args:
            vec: array-like with the values for the configuration.
            'fitness', 'n_evals', 't_pres', 'par_heter': values not present
                in 'vec' that will be appendend to the configuration dict.
        Returns:
            cfg: a configuration dictionary with the values in vec.
            
        The order of the values in vec should be that of 'par_list'.
    """
    cfg = ref_cfg
    for val, name in zip(vec, par_list):
        cfg[name] = val
    cfg['fitness'] = fitness
    cfg['n_evals'] = n_evals
    cfg['t_pres'] = t_pres
    cfg['par_heter'] = par_heter
    return cfg


In [ ]:
###############################
###### The PSO algorithm ######
###############################

W = 0.3 # inertia weight
c1 = 0.5 # weight of accel towards personal best
c2 = 0.2 # weight of accel towards global best
mv = 0.3 # maximum velocity (relative to range witdth of paramter)

use_gene = True # insert particles from a concomitant genetic algorithm
sig = lambda f: 1./(1. + np.exp(-4.*(f + 0.1))) # to set probability of insertion

max_iters = 15 # maximum number of iterations
n_procs = 15 # number of processes to use for fitness evaluation

g_best_f = 1e10 # best fitness so far
p_best_fs = [1e10] * len(pop) # best personal fitnesses
t_pres = pop[0]['t_pres'] # assuming all presentation times are equal
vels = np.random.random((len(pop), len(par_list))) - 0.5 # initial velocities

g_best = cfg_to_vec(pop[0]) # arbitrary initialization of global best
p_bests = np.zeros((len(pop), len(par_list)))
for idx, cfg in enumerate(pop):
    p_bests[idx,:] = cfg_to_vec(cfg)

# setting name for file where parameters will be stored
#path = "/home/z/projects/draculab/saves/"
path = "/home/z/Dropbox (OIST)/saves/"
fname1 = "pso"
fname2 = datetime.now().strftime('%Y-%m-%d')
fname = path + fname1 + "_" + fname2

for itr in range(max_iters):
    start_time = time.time()
    
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
        ######## parallel version, multiprocessing module
#     with Pool(n_procs) as p:
#         fits = list(p.map(eval_config, pop))
#         p.close()
#         p.join()
    ######## parallel version, pathos
    with ProcessingPool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
    #print(fits)
    # 1.2) update the average fitness values
    # In PSO configurations change every iteration, so n_evals is 
    # either 0 or 1. However, some configuration may have come from the
    # genetic algorithm, so we consider the case n_evals > 0
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        if nr > 0:
            if nr <= max_evals: 
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
        
    # 2) Find indexes that sort according to fitness. Lowest error first.
    srt_idx = np.argsort(fits)
    
    # 3.1) Update g_best
    if fits[srt_idx[0]] < g_best_f:
        g_best_f = fits[srt_idx[0]]
        g_best = cfg_to_vec(pop[srt_idx[0]])
        
    for idx, cfg in enumerate(pop):
        cfg_vec = cfg_to_vec(cfg)
        # 3.2) Update p_bests
        if cfg['fitness'] < p_best_fs[idx]:
            p_best_fs[idx] = cfg['fitness']
            p_bests[idx] = cfg_vec
            
        # 4) Update velocities
        r1, r2 = np.random.random(2)
        vels[idx] = add_vel(W * vels[idx], c1 * r1 * (p_bests[idx] - cfg_vec))
        vels[idx] = add_vel(vels[idx], c2 * r2 * (g_best - cfg_vec))
    
    # 5) Save current iteration
    best_pop = [vec_to_cfg(g_best, fitness=g_best_f, n_evals=1, t_pres=t_pres)] + pop
    with open(fname, 'wb') as f:
        pickle.dump(best_pop, f)
        f.close()
    
    # A quick message
    print("Iteration %d evaluated. Best fitness: %.3f"%(itr, g_best_f))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if g_best_f < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    
    # 6) Update particles
    for idx, cfg in enumerate(pop):
        add_to_cfg(cfg, vels[idx])
        
    #---------------------------------------------------------------------------------
    # 7) Inserting a configuration from the genetic algorithm
    if use_gene:
        g_name = path + "gene_" + fname2
        try:
            with open(g_name, 'rb') as f:
                gene_pop = pickle.load(f)
                f.close()
            gene_cfg = gene_pop[0]
            in_prob = sig(g_best_f - gene_cfg['fitness']) # insertion probability
            if np.random.random() < in_prob:
                rem_idx = np.random.randint(len(pop))
                pop[rem_idx] = gene_cfg
                fits[rem_idx] = gene_cfg['fitness']
                print("inserted pop " + str(rem_idx) + "!!!")    
        except IOError:
            if itr > 2: # if the genetic algorithm should likely be done
                from warnings import warn
                warn('population ' + g_name + 'could not be imported',
                     UserWarning)
                
    print('Iteration %d finished in %s seconds' % (itr, time.time() - start_time))
            

In [ ]:
# print final population
for dic in pop[:5]:
    print('{',end='')
    for name in dic.keys():
        if name != 'fitness' or dic['fitness'] != None:
            print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

In [ ]:
fname

In [ ]:
# Save the final population
fname = "v3_nst_afx_pop"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')
with open(fname, 'wb') as f:
    pickle.dump(pop, f)
    f.close()

In [ ]:
# Load a saved population
import pickle
fname = 'v3_nst_afx_pop'
with (open(fname, "rb")) as f:
    results = pickle.load(f)
    f.close()

In [ ]:
# Test a configuation
cfg_id = 0 # index in the population for the configuration
net, pops_dict, hand_coords, m_idxs, t_pres, _  = net_from_cfg(pop[cfg_id])
pops_names = ['SF', 'SP', 'SPF', 'AL', 'AF', 'SP_CHG', 'CE', 'CI', 'M', 'ACT', 'P']
for name in pops_names:
    exec("%s = %s"% (name, str(pops_dict[name])))

start_time = time.time()
times, data, plant_data  = net.flat_run(600.)
#times, data, plant_data  = net.run(40.)
print('Execution time is %s seconds' % (time.time() - start_time))
data = np.array(data)
#Execution time is 8.687349319458008 seconds  << before sc_inp_sum_mp, flat_run(5.)

Same history as `v3_nst_afx`
...
